In [34]:
# importing the module
import cv2
import math
from win32api import GetSystemMetrics
import time
import numpy as np
import os
import cython

In [35]:
# Image path
imagePath = 'Images'
imageName = "sample3.PNG"
imagePath = os.path.join(imagePath,imageName)
resizedImageName = imageName + "_resized"
#road detection works better with images where the scale is smaller per unit pixel 
#(screen distance = lesser real world distance)
#It also works better with a different style of image
img = cv2.imread(imagePath)

In [36]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

In [37]:
@cython.cfunc
def selectFarthestPoint(p1,ptList):
    #returns either of p2 or p3 depending on which point is farthest away from p1 (mounting point)
    maxDistance = 0
    p = [0,0]
    for pt in ptList:
        d1 = abs(math.sqrt(math.pow(p1[0]-pt[0],2)+math.pow(p1[1]-pt[1],2)))
        if(d1>maxDistance):
            p[0] = pt[0]
            p[1] = pt[1]
            maxDistance = d1
    return p

In [38]:
@cython.cfunc
def rangeCheck(checkAgainst,val,tolerance=0.01):
    if(val>=checkAgainst*(1-tolerance) and val<=checkAgainst*(1+tolerance)):
        return True
    return False

In [39]:
@cython.cfunc
def cornerCheck(p1,p2):
    h,w = img.shape[:2]
    if(rangeCheck(p1[1],h)):
        if(rangeCheck(p2[0],w)):
            return (w,h)
        elif(rangeCheck(p2[0],0)):
            return (0,h)
    elif(rangeCheck(p2[1],h)):
        if(rangeCheck(p1[0],w)):
            return (w,h)
        elif(rangeCheck(p1[0],0)):
            return (0,h)
    elif(rangeCheck(p1[1],0)):
        if(rangeCheck(p2[0],w)):
            return (w,0)
        elif(rangeCheck(p2[0],0)):
            return (0,0)
    elif(rangeCheck(p2[1],0)):
        if(rangeCheck(p1[0],w)):
            return (w,0)
        elif(rangeCheck(p1[0],0)):
            return (0,0)
        
    return (-1,-1)

In [40]:
@cython.cfunc
def getResizedForDisplayImg(img):
    screen_w, screen_h = GetSystemMetrics(0), GetSystemMetrics(1)
    #print("screen size",screen_w, screen_h)
    h,w,channel_nbr = img.shape
    # img get w of screen and adapt h
    h = h * (screen_w / w)
    w = screen_w
    if h > screen_h: #if img h still too big
        # img get h of screen and adapt w
        w = w * (screen_h / h)
        h = screen_h
    w, h = w*0.9, h*0.9 # because you don't want it to be that big, right ?
    w, h = int(w), int(h) # you need int for the cv2.resize
    return cv2.resize(img, (w, h))

In [41]:
@cython.cfunc
def getMountingPoints(img):

    imageCopy = img.copy()

    # Upper and lower color limit customized for snazzy maps
    low_red = (55, 55, 255)

    # create masks
    red_mask = cv2.inRange(imageCopy, low_red, low_red)
    
    # combine masks
    kernel = np.ones((3,3), dtype=np.uint8)
    combined_mask = cv2.morphologyEx(red_mask, cv2.MORPH_DILATE,kernel)

    # convert to grayscale
    # operatedImage = cv2.cvtColor(combined_mask, cv2.COLOR_BGR2GRAY)

    # operatedImage = cv2.fastNlMeansDenoisingColored(operatedImage,None,10,10,7,21) #uncomment if you feel image is noisy (not needed)
    # setting to 32-bit floating point
    operatedImage = np.float32(combined_mask)

    # apply the cv2.cornerHarris method
    # to detect the corners with appropriate values as input parameters
    dest = cv2.cornerHarris(operatedImage, 2, 3, 0.04)

    # Results are marked through the dilated corners
    dest = cv2.dilate(dest, None)

    # draw on the output image
    imageCopy[dest > 0.01 * dest.max()]=[255, 255, 255]

    return imageCopy

In [42]:
@cython.cfunc
def cluster2Point(clusterImg, img = np.zeros(img.shape, dtype='uint8')):
# Make the corners into one point

    # innitates the the single point corner arrays
    Points = []

    # blank = np.zeros(img.shape, dtype='uint8')
    imageCopy = img.copy()
    whiteMask = np.zeros(clusterImg.shape, dtype='uint8')
    # create masks for the corner clusters
    if(len(clusterImg.shape)==2):
        white_mask = cv2.inRange(clusterImg, 255, 255)
    elif(len(clusterImg.shape)==3):
        white_mask = cv2.inRange(clusterImg, (255,255,255), (255,255,255))

    # combine masks
    kernel = np.ones((3,3), dtype=np.uint8)
    combined_mask = cv2.morphologyEx(white_mask, cv2.MORPH_DILATE,kernel)


    cnts=cv2.findContours(combined_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    for i in cnts:
        M = cv2.moments(i)
        if M['m00'] != 0:
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
        # print(f"center - {cx},{cy}")
        Points.append([cx,cy])
        imageCopy[cy, cx] = [255, 255, 255]
        # cv2.circle(imageCopy, (cx, cy), 1, (255, 255, 255), -1)
        # cv2.drawContours(blank, [i], -1, 0, -1)

    return Points, imageCopy

In [43]:
@cython.cfunc
def getRoadsnParkings(img):
    
    tempImg = img.copy()
    # define color ranges
    # blue_lower = (250,0,0)
    blue = np.array([255, 0, 0], dtype="uint8")

    # create masks
    blue_mask = cv2.inRange(img, blue, blue)

    # combine masks
    kernel = np.ones((3,3), dtype=np.uint8)
    #combined_mask = cv2.morphologyEx(blue_mask, cv2.MORPH_DILATE,kernel)


    # findcontours
    cnts=cv2.findContours(blue_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]

    mask = np.ones(img.shape[:2], dtype="uint8") * 255
    for c in cnts:
        # area = cv2.contourArea(c)
        # for eps in np.linspace(0.001, 0.005, 5):
        #     # approximate the contour
        #     peri = cv2.arcLength(c, True)
        #     approx = cv2.approxPolyDP(c, eps * peri, True)
        cv2.drawContours(tempImg, [c], -1, 0, -1)

    mask = cv2.bitwise_not(mask)

    #image = cv2.bitwise_and(img, img, mask=mask)
    return blue_mask

In [44]:
@cython.cfunc
def pointGen(source, m, l):
    # m is the slope of line, and the
    # required Point lies distance l
    # away from the source Point
    a = Point(0, 0)
    b = Point(0, 0)

    # slope is 0
    if m == 0:
        a.x = source.x + l
        a.y = source.y

        b.x = source.x - l
        b.y = source.y

    # if slope is infinite
    elif math.isfinite(m) is False:
        a.x = source.x
        a.y = source.y + l

        b.x = source.x
        b.y = source.y - l
    else:
        dx = (l / math.sqrt(1 + (m * m)))
        dy = m * dx
        a.x = source.x + dx
        a.y = source.y + dy
        b.x = source.x - dx
        b.y = source.y - dy
    
    return [[a.x,a.y],[b.x,b.y]]

In [45]:
@cython.cfunc
def detectCorner(image):

    # making a copy of the image to have the original image untouched in main loop
    imageSub = image.copy()

    # convert to gray and perform Harris corner detection
    gray = cv2.cvtColor(imageSub,cv2.COLOR_BGR2GRAY)
    gray = np.float32(gray)
    
    #~~~~~~~~~~~~~#
    #for obtaining mounting points from red buildings img
    #~~~~~~~~~~~~~#
    dst = cv2.cornerHarris(gray,2,3,0.04)

    # result is dilated for marking the corners, not important
    dst = cv2.dilate(dst,None)

    # threshold for an optimal value, it may vary depending on the image.
    imageSub[dst>0.01*dst.max()]=[0,0,255]

    return imageSub

In [46]:
@cython.cfunc
def click_Scale_points(event, x, y, flags, params):

    # checking for left mouse clicks
    if event == cv2.EVENT_LBUTTONDOWN:
        scalePoints.append((x,y))

        # displaying the coordinates on the image window
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(img, str(x) + ',' +str(y), (x,y), font,1, (255, 255, 0), 2)
        cv2.imshow('image', img)


    # checking for right mouse clicks    
    if event==cv2.EVENT_RBUTTONDOWN:
        scalePoints.append((x,y))

        # displaying the coordinates on the image window
        font = cv2.FONT_HERSHEY_SIMPLEX
        b = img[y, x, 0]
        g = img[y, x, 1]
        r = img[y, x, 2]
        cv2.putText(img, str(b) + ',' +str(g) + ',' + str(r),(x,y), font, 1,(255, 255, 0), 2)
        cv2.imshow('image', img)

In [47]:
@cython.cfunc
def getBorderContour(img):
#function runs only once
#using new bgr values for the new image in low red.

    img_c = img.copy()
    # Upper and lower color limit customized for snazzy maps
    red = (55, 55, 255)

    # create masks
    red_mask = cv2.inRange(img, red, red)
    
    # combine masks
    kernel = np.ones((3,3), dtype=np.uint8)
    combined_mask = cv2.morphologyEx(red_mask, cv2.MORPH_DILATE,kernel)

    blank = np.zeros(img.shape, dtype='uint8')

    masked = cv2.bitwise_and(img,img,mask=combined_mask)

    # findcontours
    cnts=cv2.findContours(combined_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    for c in cnts:
        # cv2.drawContours(img, [c], -1, (255,0,255), thickness=1)
        area = cv2.contourArea(c)
        if(area>200):
            for eps in np.linspace(0.001, 0.01, 10):
                # approximate the contour
                peri = cv2.arcLength(c, True)
                approx = cv2.approxPolyDP(c, eps * peri, True)
            
            # draw the approximated contour on the image  
            cv2.drawContours(blank, [approx], -1, (255,255,255), thickness=1)
            # cv2.drawContours(blank, [c], -1, 255, thickness=1)


    # cv2.imshow("image",img)
    # cv2.waitKey(0)
    
    return blank, combined_mask


In [48]:
@cython.cfunc
def drawLine(slope, point, img):
    #draws line segment infinitely from edges of image

    returnImage = np.zeros(img.shape[:2], dtype = 'uint8')

    y_lim,x_lim = returnImage.shape
    px,py,qx,qz = [-1,-1,-1,-1] #initialize variables with 

    if(slope==math.inf or -slope==math.inf):
        px = point[0]
        py = y_lim

        qx = point[0]
        qy = 0
    else:
        px = x_lim
        py = slope*x_lim + (point[1] - slope*point[0])

        qx = 0
        qy = (point[1] - slope*point[0])

    # cv2.line(returnImage, (int(px),int(py)),(int(qx), int(qy)), 255, 1)

    try:        
        cv2.line(returnImage, (int(px),int(py)),(int(qx), int(qy)), 255, 1)
    except:
        # print(slope, px, py, qx, qy)
        pass
    return returnImage  

In [49]:
@cython.cfunc
def getRoadCoverageMask(selected_edge_list, cutQuadBorder, mountingPoint):
    totalMask = np.zeros(img.shape[:2], dtype = 'uint8')
    for selected_edge_contour in selected_edge_list:
        #mask related to a single contour, to which individual line blocking masks will be OR'ed to 
        contourMask = np.zeros(img.shape[:2], dtype = 'uint8')

        oneContourPic = np.zeros(img.shape, dtype='uint8')
        cv2.drawContours(oneContourPic, [selected_edge_contour], -1, (255,255,255), thickness=1) #draws only selected contour for the loop
        oneContourPic = cv2.cvtColor(oneContourPic, cv2.COLOR_BGR2GRAY)   #convert drawn picture into grayscale

        oneContourPic = cv2.Canny(oneContourPic,50,150,apertureSize=3)

        # Apply HoughLinesP method to 
        # to directly obtain line end points
        lines = cv2.HoughLinesP(
                    oneContourPic, # Input edge image
                    1, # Distance resolution in pixels
                    np.pi/180, # Angle resolution in radians
                    threshold=11, # Min number of votes for valid line. Use lesser number of votes for smaller lines
                    minLineLength=5, # Min allowed length of line
                    maxLineGap=10 # Max allowed gap between line for joining them
                    )

        #same line is appearing in the pic twice, perhaps increase threshold (number of votes/points)
        if(lines is None):
            continue


        for points in lines:

            demo = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

            # Extracted points nested in the list
            x1,y1,x2,y2=points[0]

            # Step 4: For each line AND it with the interior corner points ??

            interiorPointCoords_1 = (x1, y1)
            interiorPointCoords_2 = (x2, y2)

            # Step 5: Isolate the points and their coordinates that intersect with the contour line
            # Step 6: Get the slope from the mounting point to those interior corners
            # Step 7: Drop a line from the interior corners to edge of image on a quad mask so you dont have to find intersection point with further edge
            
            joiningLine_1 = drawLine((mountingPoint[1] - interiorPointCoords_1[1])/(mountingPoint[0] - interiorPointCoords_1[0]), interiorPointCoords_1, img)
            joiningLine_2 = drawLine((mountingPoint[1] - interiorPointCoords_2[1])/(mountingPoint[0] - interiorPointCoords_2[0]), interiorPointCoords_2, img)
            
            Point_1 = cv2.bitwise_and(cutQuadBorder, joiningLine_1)

            PointCoords_1, _ = cluster2Point(Point_1)  #more than one cluster might form or no cluster might wrong
            # print(PointCoords_1)
            if(len(PointCoords_1)==0):  #weird case that you can't do anything about. Realistically this shouldn't exist, but it does
                continue
            if(len(PointCoords_1)==1):
                PointCoords_1 = PointCoords_1[0]
            elif(len(PointCoords_1)==2):
                PointCoords_1 = selectFarthestPoint(mountingPoint, PointCoords_1)
            elif(len(PointCoords_1)>2):
                y,x = np.nonzero(Point_1)
                y = list(y)
                x = list(x)
                pts = [(x[a], y[a]) for a in range(len(x))]
                PointCoords_1 = selectFarthestPoint(mountingPoint, pts)

            Point_2 = cv2.bitwise_and(cutQuadBorder, joiningLine_2)
            PointCoords_2, _ = cluster2Point(Point_2)  #more than one cluster might form or no cluster might wrong
            if(len(PointCoords_2)==0):  #weird case that you can't do anything about. Realistically this shouldn't exist, but it does
                continue
            if(len(PointCoords_2)==1):
                PointCoords_2 = PointCoords_2[0]
            elif(len(PointCoords_2)==2):
                PointCoords_2 = selectFarthestPoint(mountingPoint, PointCoords_2)
            elif(len(PointCoords_2)>2):
                y,x = np.nonzero(Point_2)
                y = list(y)
                x = list(x)
                pts = [(x[a], y[a]) for a in range(len(x))]
                PointCoords_2 = selectFarthestPoint(mountingPoint, pts)

            lineMask = np.zeros(img.shape[:2], dtype='uint8')
            pointSet = [interiorPointCoords_1, interiorPointCoords_2, PointCoords_2, PointCoords_1]
            # print(pointSet)
            inclusionCorner = cornerCheck(PointCoords_1, PointCoords_2)
            if(inclusionCorner[0]!=-1):
                pointSet.insert(3,inclusionCorner)


            # Step 8: Fill the quadrilateral made by the line contour, the lines from interior corners or the view quad side wall and the view quad further wall
            # There is a concern that the points are listed in not the correct order...
            blockPts = np.array( pointSet, dtype=np.int32) #original
            # blockPts = np.array( [interiorPointCoords_1, interiorPointCoords_2, PointCoords_2[0], PointCoords_1[0]], dtype=np.int32)  #working change by Atharva
            blockPts = blockPts.reshape((-1, 1, 2))
            lineMask = cv2.fillPoly(lineMask, pts=[blockPts],color=255)

            # Step 9: OR all the quadrilateral generated inside the loop on the view quad mask
            contourMask = cv2.bitwise_or(contourMask, lineMask)
        totalMask = cv2.bitwise_or(totalMask, contourMask)
    totalMask = cv2.bitwise_not(totalMask)
    return totalMask

In [50]:
@cython.cfunc
def genQuadImages(pt):
    cutQuadMask = np.zeros(img.shape[:2], dtype='uint8')
    cutQuadBorder = np.zeros(img.shape[:2], dtype='uint8')

    pt = pt.reshape((-1,1,2))
    
    cv2.fillPoly(cutQuadMask, [pt], 255)

    # findcontours
    cnts=cv2.findContours(cutQuadMask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    for c in cnts:
        for eps in np.linspace(0.001, 0.01, 10):
            # approximate the contour
            peri = cv2.arcLength(c, True)
            approx = cv2.approxPolyDP(c, eps * peri, True)
        
        # draw the approximated contour on the image  
        cv2.drawContours(cutQuadBorder, [c], -1, (255,255,255), thickness=1)
    return cutQuadMask, cutQuadBorder

In [51]:
@cython.cfunc
def main(img, mountingPoint, ALPHA, beta, road, bldg_mask, bldg_brdr):
    # loop vars
    BETA = beta*math.pi/180
    cameraRoadCoverage = np.zeros(img.shape[:2],dtype = 'uint8')

    imgCopy = img.copy()
    roadCopy = road.copy()

    # distances of closer and further edges from mounting point
    # closer_dist = heightPix[20]*math.tan(ALPHA - (phi/2))
    further_dist = heightPix[20]*math.tan(ALPHA + (phi/2))

    # slope of horizontal plane camera angle
    slope_beta = math.tan(BETA)

    # midpoints of closer and further edges
    if beta > 180:
        further_midPoint = pointGen(Point(mountingPoint[0],mountingPoint[1]), slope_beta, further_dist)[1]
    else:
        further_midPoint = pointGen(Point(mountingPoint[0],mountingPoint[1]), slope_beta, further_dist)[0]

    further_edge =(heightPix[20]*math.tan(omega/2))/math.cos(ALPHA+(phi/2))

    # Obtaining on ground triangle points
    point1 = mountingPoint
    point2 = pointGen(Point(further_midPoint[0],further_midPoint[1]), -1/slope_beta, further_edge)[1]
    point3 = pointGen(Point(further_midPoint[0],further_midPoint[1]), -1/slope_beta, further_edge)[0]
    
    # plotting the points
    pt = np.array([point1, point2, point3], np.int32)
    cameraRoadCoverage, cutQuadBorder = genQuadImages(pt)

    # code block to check if mounting point is directly viewing inside a bldg
    circleCheck = np.zeros(img.shape[:2], dtype = "uint8")
    cv2.circle(circleCheck, (mountingPoint[0], mountingPoint[1]), 3, 255, 1)
    Check_step1 = cv2.bitwise_and(circleCheck, cameraRoadCoverage)
    Check_step2 = cv2.bitwise_and(Check_step1, bldg_mask)

    nonzeroX, _ = np.nonzero(Check_step2)
    if len(nonzeroX)>0:
        return 
    # --------------------------------#

    # get building borders inside viewing quadrilateral
    selected_bldg_brdrs_gray = cv2.cvtColor(cv2.bitwise_and(bldg_brdr,bldg_brdr, mask = cameraRoadCoverage), cv2.COLOR_BGR2GRAY) # Gray

    # 
    selected_edge_list = cv2.findContours(selected_bldg_brdrs_gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    selected_edge_list = selected_edge_list[0] if len(selected_edge_list) == 2 else selected_edge_list[1]
    
    roadCoveredMask = getRoadCoverageMask(selected_edge_list, cutQuadBorder, mountingPoint)

    cameraRoadCoverage = cv2.bitwise_and(roadCopy, cameraRoadCoverage, mask = roadCoveredMask)
    

    # find the updated are of camera coverage
    cameraRoadCoverageContour = cv2.findContours(cameraRoadCoverage, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cameraRoadCoverageContour = cameraRoadCoverageContour[0] if len(cameraRoadCoverageContour) == 2 else cameraRoadCoverageContour[1]

    
    area_sum = 0
    for contour in cameraRoadCoverageContour:
        area = cv2.contourArea(contour)
        area_sum += area
    
    return area_sum, cameraRoadCoverage, point1, point2, point3, roadCoveredMask

In [52]:
maxArea: cython.double
fp1: cython.double
fp2: cython.double
fp3: cython.double
maxCameraRoadCoverage: cython.double

scalePoints: cython.int[(0)]
mountingPoints: cython.int[(0)]

theta: cython.double
phi: cython.double
omega: cython.double
alpha: cython.double

scaleConst: cython.double
scale: cython.int
heights: cython.int[0]
heightPix: cython.int[0]

In [53]:
scalePoints = [(302, 373), (464, 373)] # points obtained from sample_scale.PNG map which is a google maps with the same dimensions

maxArea = 0

theta = (66.75*math.pi)/180    #diagonal angle FOV of camera (GIVEN!!)
phi = 2*math.atan(0.8*math.tan(theta/2))  #angle of view larger side of camera resolution (4 in 4:3)
omega = 2*math.atan(0.6*math.tan(theta/2))     #angle of view larger side of camera resolution (3 in 4:3)
alpha = (75*math.pi)/180   #set later on in the code based on the height of the camera [angle of camera from negative z axis]

In [54]:
# Scale for pix to meter conversion
# scaleConst = int(input('Scale: '))
scaleConst = 20 # for the below pixel distance, this represent

# Get mounting points from building borders
# mountingPoints = get_MP_FromBuildings_ButAlsoText(img)

# Pixel distance of scale in image
# actual distance(m) = (scale constant)*(obtained magnitude)/scale
scale = math.ceil(abs(math.sqrt(pow(scalePoints[0][0] - scalePoints[1][0],2) + pow(scalePoints[0][1] - scalePoints[1][1],2))))

# heights = [x*0.1 for x in range(30,60)] #average height of light poles is 9 to 14 feet ~ 4.2m max
heights = [h*0.1 for h in range(30, 60)]    #3m to 6m
# Converting height in meter array to height in pixel array
heightPix = [h*scale/scaleConst for h in heights]

In [55]:
# reading the resized the image
# img = cv2.imread(imagePath)

# obtaining the mask of the roads and parkings in the map image
road = getRoadsnParkings(img)   #image is grayscale


In [56]:
cv2.imshow('win', road)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [57]:
img = cv2.imread(imagePath)

mountingCluster = getMountingPoints(img)
mountingPointsList, mountingPointsImg = cluster2Point(mountingCluster, img)

cv2.imshow("mounting points as pixels", mountingPointsImg)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [58]:
# getting building borders by using a mask function
bldg_brdr, bldg_mask = getBorderContour(img)

In [59]:
# angle of camera from negative of vertical axisS
img = cv2.imread(imagePath)
ALPHA = 60*math.pi/180

for mountingPoint in mountingPointsList:
    for beta in range (0,360, 10):

        # beta edge cases
        if(beta==180 or beta==0 or beta==360):
            continue

        # loop vars
        BETA = beta*math.pi/180
        cameraRoadCoverage = np.zeros(img.shape[:2],dtype = 'uint8')

        imgCopy = img.copy()
        roadCopy = road.copy()

        # distances of closer and further edges from mounting point
        # closer_dist = heightPix[20]*math.tan(ALPHA - (phi/2))
        further_dist = heightPix[20]*math.tan(ALPHA + (phi/2))

        # slope of horizontal plane camera angle
        slope_beta = math.tan(BETA)

        # midpoints of closer and further edges
        if beta > 180:
            further_midPoint = pointGen(Point(mountingPoint[0],mountingPoint[1]), slope_beta, further_dist)[1]
        else:
            further_midPoint = pointGen(Point(mountingPoint[0],mountingPoint[1]), slope_beta, further_dist)[0]

        further_edge =(heightPix[20]*math.tan(omega/2))/math.cos(ALPHA+(phi/2))

        # Obtaining on ground triangle points
        point1 = mountingPoint
        point2 = pointGen(Point(further_midPoint[0],further_midPoint[1]), -1/slope_beta, further_edge)[1]
        point3 = pointGen(Point(further_midPoint[0],further_midPoint[1]), -1/slope_beta, further_edge)[0]
        
        # plotting the points
        pt = np.array([point1, point2, point3], np.int32)
        cameraRoadCoverage, cutQuadBorder = genQuadImages(pt)

        # code block to check if mounting point is directly viewing inside a bldg
        circleCheck = np.zeros(img.shape[:2], dtype = "uint8")
        cv2.circle(circleCheck, (mountingPoint[0], mountingPoint[1]), 3, 255, 1)
        Check_step1 = cv2.bitwise_and(circleCheck, cameraRoadCoverage)
        Check_step2 = cv2.bitwise_and(Check_step1, bldg_mask)

        nonzeroX, _ = np.nonzero(Check_step2)
        if len(nonzeroX)>0:
            continue
        # --------------------------------#

        # get building borders inside viewing quadrilateral
        selected_bldg_brdrs_gray = cv2.cvtColor(cv2.bitwise_and(bldg_brdr,bldg_brdr, mask = cameraRoadCoverage), cv2.COLOR_BGR2GRAY) # Gray

        # 
        selected_edge_list = cv2.findContours(selected_bldg_brdrs_gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        selected_edge_list = selected_edge_list[0] if len(selected_edge_list) == 2 else selected_edge_list[1]
        
        roadCoveredMask = getRoadCoverageMask(selected_edge_list, cutQuadBorder, mountingPoint)

        cameraRoadCoverage = cv2.bitwise_and(roadCopy, cameraRoadCoverage, mask = roadCoveredMask)
        

        # find the updated are of camera coverage
        cameraRoadCoverageContour = cv2.findContours(cameraRoadCoverage, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cameraRoadCoverageContour = cameraRoadCoverageContour[0] if len(cameraRoadCoverageContour) == 2 else cameraRoadCoverageContour[1]

        
        area_sum = 0
        for contour in cameraRoadCoverageContour:
            area = cv2.contourArea(contour)
            area_sum += area

        if area_sum > maxArea:
            maxArea = area_sum
            fp1 = point1
            fp2 = point2
            fp3 = point3
            mp = mountingPoint
            excludedRoad = roadCoveredMask
            maxCameraRoadCoverage = cv2.bitwise_or(cv2.cvtColor(imgCopy, cv2.COLOR_BGR2GRAY), cameraRoadCoverage)

        

try:
    # plotting best shape and the mounting point
    pts = np.array([fp1, fp2, fp3], np.int32)
    pts = pts.reshape((-1,1,2))
    cv2.polylines(img, [pts], True, (255,255,0))
    cv2.circle(img, (mp[0], mp[1]), 5, (255,0,0),2)


    cv2.polylines(maxCameraRoadCoverage, [pts], True, (255,255,0))
    cv2.circle(maxCameraRoadCoverage, (mp[0], mp[1]), 5, (255,0,0),2)
except:
    print(fp1, fp2, fp3)



C:\Users\Salil kulkarni\AppData\Local\Temp\ipykernel_676\757620607.py:46: RuntimeWarning: divide by zero encountered in int_scalars
  joiningLine_1 = drawLine((mountingPoint[1] - interiorPointCoords_1[1])/(mountingPoint[0] - interiorPointCoords_1[0]), interiorPointCoords_1, img)
C:\Users\Salil kulkarni\AppData\Local\Temp\ipykernel_676\757620607.py:47: RuntimeWarning: divide by zero encountered in int_scalars
  joiningLine_2 = drawLine((mountingPoint[1] - interiorPointCoords_2[1])/(mountingPoint[0] - interiorPointCoords_2[0]), interiorPointCoords_2, img)
C:\Users\Salil kulkarni\AppData\Local\Temp\ipykernel_676\757620607.py:46: RuntimeWarning: invalid value encountered in int_scalars
  joiningLine_1 = drawLine((mountingPoint[1] - interiorPointCoords_1[1])/(mountingPoint[0] - interiorPointCoords_1[0]), interiorPointCoords_1, img)
C:\Users\Salil kulkarni\AppData\Local\Temp\ipykernel_676\757620607.py:47: RuntimeWarning: invalid value encountered in int_scalars
  joiningLine_2 = drawLine((mo

In [60]:
# showing image
cv2.imshow('included', maxCameraRoadCoverage)
# cv2.imshow('excluded', GlobalCheck1)
cv2.waitKey(0)
cv2.destroyAllWindows()